In [ ]:
import autograd.numpy as np
import autograd.numpy.random as npr

In [22]:
def simulate_LDA(K, V, N, M, ETA=0.1, ALPHA=0.5, rs_int=npr.randint(low=0, high=100)):
    rs = npr.RandomState(rs_int)
    BETA = rs.dirichlet(np.full(V, ETA), size=K)
    THETA = rs.dirichlet(np.full(K, ALPHA), size=N)
    X = []
    for i in range(N):
        x_i = np.zeros(M[i], dtype=int)
        for j in range(M[i]):
            z_ij = rs.choice(K, p=THETA[i])
            x_ij = rs.choice(V, p=BETA[z_ij])
            x_i[j] = x_ij
        X.append(x_i)
    return X

def init_variational_params(X, K, V):
    N = len(X)
    M = np.array([len(x_i) for x_i in X])
    LAMBDA = npr.uniform(low=0.01, high=1.0, size=(K, V))
    GAMMA = np.ones((N, K))
    PHI = []
    for M_i in M:
        PHI_i = np.ones((M_i, K))
        PHI_i = PHI_i / K
        PHI.append(PHI_i)
    return (LAMBDA, GAMMA, PHI)

rs = npr.RandomState(0)
K = 10
V = 500
N = 50
M = rs.poisson(75, size=N)
X = simulate_LDA(K, V, N, M)

In [ ]:
def unpack_params(params):
    LAMBDA, GAMMA, PHI = params[0], params[1], params[2]
    return LAMBDA, GAMMA, PHI

def sample_variational_params(variational_params, num_samples=100):
    LAMBDA, GAMMA, PHI = unpack_params(variational_params)
    K, V = LAMBDA.shape
    N, _ = GAMMA.shape
    M = np.array([len(PHI_doc) for PHI_doc in PHI])